1.Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

2.Read the file ( Dask, Modin, Ray, pandas and present your findings in term of computational efficiency )

3.Perform basic validation on data columns : eg: remove special character , white spaces from the col name

4.As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of
read and write file, column name in YAML

5.Validate number of columns and column name of ingested file with YAML.

6.Write the file in pipe separated text file (|) in gz format.

7.Create a summary of the file:

Total number of rows,

total number of columns

file size

1.Take any csv/text file of 2+ GB of your choice. 

I have found a 3.49GB file on kaggle: https://www.kaggle.com/code/souamesannis/tips-to-work-efficiently-with-the-dataset/data?select=transactions_train.csv

2.Read the file ( Dask, Modin, Ray, pandas and present your findings in term of computational efficiency )


In [ ]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import re
import yaml

In [10]:
%%time
data_pandas = pd.read_csv('transactions_train.csv')

CPU times: total: 1min 27s
Wall time: 1min 33s


In [15]:
%%time
data_dask = dd.read_csv('transactions_train.csv')

CPU times: total: 46.9 ms
Wall time: 79.4 ms


In [16]:
%%time 
data_dask.compute()

CPU times: total: 2min 8s
Wall time: 46.3 s


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
588586,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
588587,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
588588,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
588589,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [18]:
data_dask.npartitions

54

In [37]:
import os

os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray

import modin.pandas as pd
import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2022-11-07 19:07:47,524	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.11
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8266


In [38]:
%%time 
data_modin_ray = mpd.read_csv('transactions_train.csv')

CPU times: total: 14.7 s
Wall time: 1min 1s


In [39]:
import os


os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

import modin.pandas as pd

In [40]:
%%time 
data_modin_dask = mpd.read_csv('transactions_train.csv')

CPU times: total: 15.1 s
Wall time: 53.2 s


Times:
pandas - 1min 33sec
dask - 46.3sec
modin-ray  - 1min 1sec
modin-dask - 53.2sec

3.Perform basic validation on data columns : eg: remove special character , white spaces from the col name

In [105]:
def column_names_normalisation(columns):
    columns = columns.str.replace(' ','_',regex=True)
    columns = list(map(lambda x: re.sub('\W+','', x),list(columns)))
    columns = list(map(lambda x: x.strip('_'), list(columns)))
    columns =list(map(lambda x: x.lower(), list(columns)))
    return columns

In [106]:
data_pandas.columns = column_names_normalisation(data_pandas.columns)
data_pandas.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id'], dtype='object')

4.As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of
read and write file, column name in YAML


In [42]:
%%writefile file.yaml
file_type: csv
dataset_name: data_pandas
file_name: transactions_train
table_name: transactions_train
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - t_dat
    - customer_id
    - article_id
    - price
    - sales_channel_id

Writing file.yaml



5.Validate number of columns and column name of ingested file with YAML.



In [131]:
#I think think this should be done at the beginning of the task and not after the column normalisation,
#so I'll use original columns


with open('file.yaml','r') as yaml_file:
    yml_file = yaml.safe_load(yaml_file)

expected_cols = yml_file['columns']
columns = list(data_pandas.columns)

def check_column_match(columns, expected_cols):

    if len(columns) == len(expected_col) and list(set(columns).difference(expected_col)) == []:
        print("column name and column length validation passed")
        
    else:
        print("column name and column length validation failed")
        
        missing_from_yaml = list(set(columns).difference(expected_col))
        print("Following File columns are not in the YAML file",missing_from_yaml)
        
        missing_from_data = list(set(expected_col).difference(columns))
        print("Following YAML columns are not in the file uploaded",missing_from_data)

check_column_match(columns, expected_cols)

column name and column length validation passed


6.Write the file in pipe separated text file (|) in gz format.



In [137]:
data_pandas.to_csv("transactions_train.txt.gz", 
           index=False, 
           sep='|',
           compression="gzip")

In [140]:
data_pandas.shape[0]

31788324

7.Create a summary of the file:

Total number of rows,

total number of columns

file size

In [150]:
import enum
class SIZE_UNIT(enum.Enum):
   BYTES = 1
   KB = 2
   MB = 3
   GB = 4
    
def convert_unit(size_in_bytes, unit):
   """ Convert the size from bytes to other units like KB, MB or GB"""
   if unit == SIZE_UNIT.KB:
       return size_in_bytes/1024
   elif unit == SIZE_UNIT.MB:
       return size_in_bytes/(1024*1024)
   elif unit == SIZE_UNIT.GB:
       return size_in_bytes/(1024*1024*1024)
   else:
       return size_in_bytes

In [151]:
import os
def get_file_size(file_name, size_type = SIZE_UNIT.BYTES ):
   """ Get file in size in given unit like KB, MB or GB"""
   size = os.path.getsize(file_name)
   return convert_unit(size, size_type)

In [159]:
file_path = 'transactions_train.txt.gz'
# get file size in KB
size = get_file_size(file_path, SIZE_UNIT.MB)

In [161]:
print(f'Total number of rows: {data_pandas.shape[0]}\nTotal number of columns: {data_pandas.shape[1]} \nFIle size: {size} MB')

Total number of rows: 31788324
Total number of columns: 5 
FIle size: 568.0889711380005 MB
